In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [18]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 600)

In [40]:
s = requests.Session()

In [67]:
keyword = 'radio'

In [68]:
r = s.get(f'https://garuda.kemdikbud.go.id/documents?q={keyword}')
res = BeautifulSoup(r.text, 'html.parser')

In [69]:
# res.find('p', attrs={'class':'pagination-info'}).text.split('|')[0].strip().split(' ')[-1]
pagesize = res.find('p', attrs={'class':'pagination-info'}).text.split('|')[0].strip().split(' ')[-1]
jumlah_paper = res.find('p', attrs={'class':'pagination-info'}).text.split('|')[1].replace(' Total Record : ', '').strip()
pages = int(res.find('p', attrs={'class':'pagination-info'}).text.split('|')[0].replace('Page 1 of ', '').strip())
print(f'Jumlah Laman: {pages}\nJumlah Paper: {jumlah_paper}')

Jumlah Laman: 298
Jumlah Paper: 2976


In [70]:
results = []

total_page = pages+1
# for page in range(1):
for page in tqdm(range(1, total_page)):
  r = s.get(f'https://garuda.kemdikbud.go.id/documents?page={page}&q={keyword}&select=title')
  res = BeautifulSoup(r.text, 'html.parser')
  articles = res.findAll('div', attrs={'class':'article-item'})
  for item in articles:
    title = item.find('a', attrs={'class':'title-article'}).text.strip()
    article_id = item.find('a', attrs={'class':'title-article'})['href'].split('/')[-1]
    author_ids = '|'.join(a['href'].split('/')[-1] for a in item.find_all('a', attrs={'class':'author-article'}))
    author_name = '|'.join(a.text for a in item.find_all('a', attrs={'class':'author-article'}))
    journal = item.find_all('xmp', attrs={'class':'subtitle-article'})[0].text.strip()
    publisher = item.find_all('xmp', attrs={'class':'subtitle-article'})[1].text.strip()
    abstract = item.find('xmp', attrs={'class':'abstract-article'}).text.strip()

    title_citation = item.find_all('a', attrs={'class':'title-citation'})
    for a in title_citation:
      pdf_ori = a['href'] if 'Download' in a.text else ''
      source_ori = a['href'] if 'Source' in a.text else ''
      pdf_garuda = a['href'] if 'PDF' in a.text else ''
      doi = a['href'] if 'DOI' in a.text else ''

    data = {'article_id':article_id, 'title':title, 'author_name':author_name, 'author_ids':author_ids, 
            'journal':journal, 'publisher':publisher, 'abstract':abstract,
            'source_ori': source_ori, 'pdf_ori': pdf_ori, 'pdf_garuda': pdf_garuda, 'doi': doi}
    
    results.append(data)

100%|██████████| 298/298 [03:01<00:00,  1.64it/s]


In [71]:
len(results)

2976

In [72]:
df = pd.DataFrame(results)
df

,article_id,title,author_name,author_ids,journal,publisher,abstract,source_ori,pdf_ori,pdf_garuda,doi
0,1462389,PENYIDIKANTERHADAP TINDAK PIDANA PENGGUNAAN SP...,Roni Gunawan Rajagukguk|Erdianto Effendi|Ledy ...,3228023|462329|253711,Jurnal Online Mahasiswa (JOM) Bidang Ilmu Huku...,Jurnal Online Mahasiswa (JOM) Bidang Ilmu Hukum,Private radio stations that do not have radio ...,,,,
1,785024,PENYIDIKANTERHADAP TINDAK PIDANA PENGGUNAAN SP...,Roni Gunawan Rajagukguk|Erdianto '|Ledy Diana,3228023|3223381|253711,Jurnal Online Mahasiswa (JOM) Bidang Ilmu Huku...,Jurnal Online Mahasiswa (JOM) Bidang Ilmu Hukum,Private radio stations that do not have radio ...,,,,
2,1698420,National University Online Radio Using Shoutca...,Rafli Ramadhan Gunawan|Septi Andryana|Aris Gun...,3998526|500897|2406094,Jurnal Mantik Vol. 4 No. 1 (2020): May: Manaje...,Institute of Computer Science (IOCS),"Today's technological developments, where work...",,,http://download.garuda.kemdikbud.go.id/article...,
3,2761655,KOHESI LEKSIKAL DAN JENIS TEMA PADA SPIRIT HAR...,Anisak Syaid Fauziah|Sumarlam Sumarlam|Djatmik...,3670249|2430707|3670250,"Metalingua: Jurnal Penelitian Bahasa Vol 17, N...",Balai Bahasa Jawa Barat,AbstractThe article discusses lexical cohesion...,,,,https://doi.org/10.26499/metalingua.v17i1.288
4,2993924,IMPLEMENTASI PENERIMA RADIO FM (FM RADIO RECEI...,Iswanto Iswanto,4179194,Jurnal Teknologi Informasi dan Komunikasi Vol....,Fakultas Ilmu Komputer dan Informatika - Unive...,Pertukaran data analog dan digital menggunakan...,,,,https://doi.org/10.56244/fiki.v11i1.426
...,...,...,...,...,...,...,...,...,...,...,...
2971,3949324,PEngaruh Iklan Obat di Televisi dan Radio pada...,Johannes Hudyono,469299,Jurnal Kedokteran Meditek Vol. 6 No. 17 Oktobe...,Fakultas Kedokteran Universitas Kristen Krida ...,,,,,https://doi.org/10.36452/jkdoktmeditek.v6i17.943
2972,3952351,Kontribusi Radio City Guide 911 FM Mewujudkan ...,Hasna Fadhilah,6803615,Journal of Islamic Business Law Vol 2 No 4 (20...,Fakultas Syariah Universitas Islam Negeri Maul...,Penelitian ini bertujuan untuk mengetahui kont...,,,,
2973,3953516,Analisa Dan Desain Sistem Penunjang Keputusan ...,Rania Khalda Abdiansyah|Lis Suryadi,6805403|2885580,Prosiding Seminar Nasional Unimus Vol 4 (2021)...,Universitas Muhammadiyah Semarang,Sistem ini membahas tentang keputusan seleksi ...,,,,
2974,3953562,SOLAR CELL INTEGRATION WITH RADIO WAVE AI FOR ...,Kelvin Smith,6805469,BULLET : Jurnal Multidisiplin Ilmu Vol. 3 No. ...,CV. Multi Kreasi Media,Within the rapidly expanding field of sustaina...,,,,


In [73]:
df.to_csv(f'data/garuda_{keyword}.csv', index=False)

### END HERE

In [10]:
r = s.get(f'https://garuda.kemdikbud.go.id/documents?q={keyword}')
res = BeautifulSoup(r.text, 'html.parser')

In [52]:
for item in articles:
    journal = item.find_all('xmp', attrs={'class':'subtitle-article'})[0].text.strip()
    author_ids = '|'.join(a['href'].split('/')[-1] for a in item.find_all('a', attrs={'class':'author-article'}))
    author_name = '|'.join(a.text for a in item.find_all('a', attrs={'class':'author-article'}))

    print(author_name)

Mohamad Yudharistyawan Widodo
Said Firdaus |Zuraidah - |Febyola Rafiq
Dimas Aditya Nugraha
Gunawan Kusmantoro
Ahmad Budiman
Kristopo |Bachtiar, Alfan |Supriyanto, Bambang Heri
Widyatmi Anandy |Irzha Friskanov. S
Ahmad Budiman
Ayu Permata Sari*, Suradi, Aminah
Makawimbang, Halmintje |Rumapea, Patar |Ruru, Joorie


In [ ]:


articles = res.findAll('div', attrs={'class':'article-item'})
for item in articles:
    title_citation = item.find_all('a', attrs={'class':'title-citation'})
    for a in title_citation:
        pdf_ori = a['href'] if 'Download' in a.text else ''
        source_ori = a['href'] if 'Source' in a.text else ''
        pdf_garuda = a['href'] if 'PDF' in a.text else ''
        doi = a['href'] if 'DOI' in a.text else ''

        # print(a) # Download Original, Original Source, Check in Google Scholar, Full PDF